In [12]:
!pip install tqdm tensorflow-hub tensorflow-text

/bin/bash: /home/vedantjumle/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Obtaining dependency information for tensorflow-text from https://files.pythonhosted.org/packages/a9/48/7749e08961469a729a6c11a59265f648849e3f7fc69be93131389a9fe31d/tensorflow_text-2.13.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 71.7 MB/s eta 0:00:0000:0100:01


In [1]:
import pandas as pd
import json
import os
from tqdm import tqdm
from collections import defaultdict

In [2]:
def load_jsonl(filepath):
    with open(filepath, 'r', encoding='utf-8') as json_file:
        json_list = list(json_file)

    jsons = []

    for json_str in json_list:
        jsons.append(json.loads(json_str))

    return jsons

In [3]:
root_dir = "./MASSIVE/"

In [4]:
train_filelist = os.listdir(f"{root_dir}/train_data")
test_filelist = os.listdir(f"{root_dir}/test_data")
len(train_filelist), len(test_filelist)

(52, 51)

In [5]:
train_texts = []
train_labels = []

for item in train_filelist:
    data = load_jsonl(f"{root_dir}/train_data/{item}")

    for example in tqdm(data, f"loading {item}"):
        train_texts.append(example["utt"])
        train_labels.append(example["intent"])

loading pt-PT.jsonl: 100%|███████████████████████████████████████████████████| 16521/16521 [00:00<00:00, 992595.67it/s]


In [6]:
from tensorflow.keras.utils import to_categorical
from pandas import Series

2023-08-25 07:52:43.817710: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-25 07:52:44.493975: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-25 07:52:46.164947: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
unq = Series(train_labels).unique()
idx = Series(Series(train_labels).unique()).index
mapping = Series(idx, index=unq)
train_labels = to_categorical([mapping[item] for item in train_labels])

In [8]:
train_labels.shape

(859092, 60)

In [9]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [13]:
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text  # Needed for loading universal-sentence-encoder-cmlm/multilingual-preprocess
import numpy as np

preprocessor = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
encoder = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-base-br/1", trainable=False)

# english_embeds = encoder(preprocessor(english_sentences))["default"]
# japanese_embeds = encoder(preprocessor(japanese_sentences))["default"]
# italian_embeds = encoder(preprocessor(italian_sentences))["default"]

2023-08-25 07:53:29.858493: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20502 MB memory:  -> device: 0, name: NVIDIA L4, pci bus id: 0000:00:03.0, compute capability: 8.9


In [14]:
BATCH_SIZE = 64

dataset = (tf.data.Dataset.from_tensor_slices((train_texts, train_labels))
            .shuffle(buffer_size=1000)
            .batch(BATCH_SIZE))

DATASET_SIZE = len(dataset)

split = 0.8

train_size = int(split * DATASET_SIZE)
val_size = int(1-split * DATASET_SIZE)

train_ds = dataset.take(train_size)
validation_ds = dataset.skip(train_size)

len(train_ds), len(validation_ds)

(10739, 2685)

In [15]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from pandas import Series

In [16]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
x = preprocessor(text_input)
x = encoder(x)["default"]
x = layers.Dense(train_labels.shape[-1], activation='softmax')(x)

model = Model(inputs=text_input, outputs=x)
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['input_1[0][0]']             
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [17]:
# ModelCheckpoint: Save the model's weights after every epoch
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='./checkpoints/model_checkpoint.h5', save_best_only=True)
# EarlyStopping: Stop training when a monitored metric has stopped improving
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [18]:
history = model.fit(train_ds, validation_data=validation_ds, epochs=5, callbacks=[model_checkpoint, early_stopping])

Epoch 1/5


2023-08-25 07:55:29.626815: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-08-25 07:55:30.432875: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x393a8780 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-25 07:55:30.432907: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA L4, Compute Capability 8.9
2023-08-25 07:55:30.527435: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-25 07:55:31.034768: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-25 07:55:31.126832: W tensorflow/compiler/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:543] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in

InternalError: Graph execution error:

Detected at node 'Adam/StatefulPartitionedCall_1' defined at (most recent call last):
    File "/home/vedantjumle/miniconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/vedantjumle/miniconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/vedantjumle/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/vedantjumle/miniconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/vedantjumle/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 736, in start
      self.io_loop.start()
    File "/home/vedantjumle/miniconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio